In [2]:
import datetime
import time
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import tushare as ts
import dateutil


In [16]:
'''
1.数据获取
'''
# 读取数据库数据
def read_from_datebase(stock_code:str, start_date=None, end_date=None):
    # 数据库路径
    datebase = 'database'
    # 将字符串转换为时间格式
    start_date = datetime.strptime(start_date, '%Y%m%d') if start_date else datetime.strptime('20240101', '%Y%m%d')
    end_date = datetime.strptime(end_date, '%Y%m%d') if end_date else datetime.strptime('20241231', '%Y%m%d')
    # 读取数据
    data = pd.read_csv(datebase + '/' + stock_code + '.csv')
    # 将时间列转换为时间格式
    data['trading_time'] = pd.to_datetime(data['trading_time'], format='%Y%m%d%H%M%S%f')
    # 将时间列设置为索引
    data.set_index('trading_time', inplace=True)
    # 筛选数据
    data = data.loc[(data.index >= start_date) & (data.index <= end_date)]
    return data

# 获取交易日历
def get_trade_calendar(stock_code):
    # 读取数据库数据并获取索引作为交易日历
    trade_calendar = read_from_datebase(stock_code).index.to_series()
    return trade_calendar

# 获取tick数据
def get_tick_data(stock_codes):
    ts.set_token('567cc6a3f980227a5844d977ee1e53b96555778bb3d038a0613f7699')
    df = ts.realtime_quote(ts_code=stock_codes)
    return df

In [18]:
stock_codes = '600519.SH,000001SH'
data = get_tick_data(stock_codes)
data

,NAME,TS_CODE,DATE,TIME,OPEN,PRE_CLOSE,PRICE,HIGH,LOW,BID,...,A1_V,A1_P,A2_V,A2_P,A3_V,A3_P,A4_V,A4_P,A5_V,A5_P
0,贵州茅台,600519.SH,20240419,15:00:00,1660.78,1670.78,1646.64,1668.5,1641.28,1646.6,...,0.0,1646.64,2,1647.0,1,1647.5,17,1648.0,8,1648.01


In [ ]:
'''
2.Context类(上下文数据)
'''
class Context:
    def __init__(self, run_type:str, stock_list:list, cash:int, start_date:str, end_date:str, trade_cal:pd.Series):
        # 运行类型: 'backtest'回测, 'paper'模拟交易
        self.run_type = run_type
        # 初始现金
        self.cash=cash
        # 开始时间
        self.start_date = datetime.strptime(start_date, '%Y-%m-%d')
        # 结束时间
        self.end_date = datetime.strptime(end_date, '%Y-%m-%d')
        '''
        持仓
        positions的数据结构
        {
            'stock_codes': volume,
            ...
        }
        其中: stock_codes为品种名称, volume为持仓量
        '''
        self.positions = {stock_code:0 for stock_code in stock_list}
        # 基准
        self.benchmark = None
        # 回测交易日历
        self.date_range = trade_cal.loc[self.start_date:self.end_date].values
        # 当前时间，起始日期设为start_date，随进程更新
        self.dt = datetime.strptime(start_date, '%Y-%m-%d')
        # 交易账单
        self.trade_log = pd.DataFrame(columns=['stock_code', 'volume', 'price'])


# 全局变量
class G:
    pass
g = G()

# 实例化上下文数据context
# 初始资金cash，开始时间，结束时间
context = Context(cash = 100000, start_date = '2024-01-02', end_date = '2024-01-03', trade_cal = get_trade_calendar('ao'))

# 设置基准benchmark
def set_bench_mark(stock_codes):
    context.benchmark = stock_codes

# 历史行情数据获取
def get_ohlc(stock_codes, curr_time):
    # 获取所有数据
    all_data = read_from_datebase(stock_codes, ['open', 'close', 'high', 'low', 'volume'])
    
    # 判断当前时间是否在交易时间内
    if curr_time not in context.date_range:
        return pd.DataFrame(columns=['open', 'close', 'high', 'low', 'volume'], index=[curr_time])
    
    realtime_data = all_data.loc[curr_time]
    return realtime_data

In [ ]:
'''
3.交易相关订单函数
'''
# 买卖订单基础函数
def _order(stock_code, price, volume):
    # 现金不足
    if context.cash - volume * price < 0:
        volume = int(context.cash / price / 100) * 100
        print(f"现金不足，已调整为{volume}")
        
    # 100的倍数
    if volume % 100 != 0:
        if volume != -context.positions.get(stock_code, 0):
            volume = int(volume / 100) * 100
            print(f"不是100的倍数, 已调整为{volume}")
            
    # 卖出数量超过持仓数量
    if context.positions.get(stock_code, 0) < -volume:
        volume = -context.positions.get(stock_code,0)
        print(f"卖出数量不能超过持仓数量, 已调整为{volume}")
        
    # 将买卖股票数量存入持仓标的信息
    context.positions[stock_code] = context.positions.get(stock_code, 0) + volume
    
    # 剩余资金
    context.cash -= volume * price
    # 交易记录
    context.trade_log.loc[context.dt] = [stock_code, volume, price]
    return volume
    
# 按股数下单
def order(stock_code, volume):
    # 获取实时行情
    _order(stock_code, get_tick_data(stock_code)['PRICE'], volume)

# 目标股数下单
def order_target(stock_code, volume):
    if volume < 0:
        print("数量不能为负,已调整为0")
        volume = 0
    # 当前持有数量
    hold_volume = context.positions.get(stock_code, 0)
    # 交易数量
    delta_volume = volume - hold_volume
    _order(stock_code, get_tick_data(stock_code)['PRICE'], delta_volume)

# 按价值下单
def order_value(stock_code, value):
    _order(stock_code, get_tick_data(stock_code)['PRICE'], int(value / get_tick_data(stock_code)['PRICE']))

# 目标价值下单
def order_target_value(stock_code, value):
    if value < 0:
        print("价值不能为负,已调整为0")
        value = 0
    hold_value = context.positions.get(stock_code, 0) * get_tick_data(stock_code)['PRICE']
    delta_value = value - hold_value
    order_value(stock_code, delta_value)

# twap下单
# twap_param为拆分份数
def order_twap(stock_code, volume, twap_shares):
    target_volume = int(volume / twap_shares)
    # 已成交量
    traded_volume = 0
    # 下单twap_param次
    for i in range(twap_shares):
        # 下单
        order_volume = order(stock_code, target_volume)
        # 如果下单数量为0，说明现金不足，结束本次交易
        if order_volume == 0:
            print("剩余现金不足")
            break
        # 已成交数量
        traded_volume += order_volume
        # 重新计算下单数量
        target_volume = int((volume - traded_volume) / (twap_shares - i))
        # 休眠
        time.sleep(60)

# 获取同时段前n个交易日的数据
def get_prev_data(stock_code, curr_time, vwap_shares=15, back_days=5):
    port_data = pd.DataFrame()
    i = 1
    while len(prev_data) < back_days:
        # 获取前i个交易日的数据
        prev_time = curr_time - pd.tseries.offsets.BDay(i)
        # 如果前i个交易日在交易日历内
        if prev_time in context.date_range:
            # 获取前i个交易日的数据
            prev_time_index = context.date_range.get_loc(prev_time)
            # 获取vwap_shares份交易的数据
            prev_data = read_from_datebase(stock_code, ['volume']).iloc[prev_time_index: prev_time_index + vwap_shares].copy()
            # 计算交易量占比
            prev_data['port'] = prev_data['volume'] / prev_data['volume'].sum()
            # 将数据拼接
            port_data = pd.concat([port_data, prev_data], axis=1, ignore_index=True)
        i += 1
    port_data['port_mean'] = port_data.mean(axis=1)
    return port_data['port_mean']

# vwap下单
def order_vwap(stock_code, volume, vwap_shares=15, back_days=5):
    curr_time = datetime.now()
    port_mean = get_prev_data(stock_code, curr_time, vwap_shares, back_days)
    traded_volume = 0
    delta_volume = 0
    # 计算当前时间前七个交易日的同时间段的交易量占比作为当前交易份额
    for i in range(vwap_shares):
        target_volume = int(volume * port_mean[i]) + delta_volume
        # 下单
        order_volume = order(stock_code, target_volume)
        # 如果下单数量为0，说明现金不足，结束本次交易
        if order_volume == 0:
            print("剩余现金不足")
            break
        # 已成交数量
        traded_volume += order_volume
        # 本次未成交数量，进入下次交易
        delta_volume = target_volume - order_volume
        # 休眠
        time.sleep(60)


In [ ]:
'''
4.策略回测层(主函数)
'''

# 框架主体函数
def run():
    # 创建收益数据表
    plt_df = pd.DataFrame(index=context.date_range, columns=['value'])
    # 初始资金
    init_value = context.cash
    # 初始化函数
    initialize(context)
    last_price = {}
    # 模拟每个bar运行
    for dt in context.date_range:
        # 将context对象中的日期context.dt更新为当前迭代的日期dt。这是在模拟回测过程中更新当前日期的操作
        context.dt = dt
        # 调用handle_data()函数，传递context对象作为参数，执行具体的交易逻辑处理
        handle_data(context)
        # 将当前资金的数值保存到value变量中
        value = context.cash
        # 遍历每支股票计算股票价值
        for stock in context.positions:
            today_data = get_ohlc(stock, context.dt)
            # 停牌
            if len(today_data) == 0:
                # 停牌前一个交易日价格
                p = last_price[stock]
            else:
                p = today_data['open']
                #存储为停牌前一个交易日价格
                last_price[stock] = p
            # 计算当前持仓股票的总价值
            value += p * context.positions[stock]
        # 将当前日期dt对应的总资产价值value，保存到收益数据表plt_df中
        plt_df.loc[dt, 'value'] = value
    #计算投资组合价值相对于初始值的收益率，并将结果存储在名为ratio 的新列中
    plt_df['ratio'] = (plt_df['value'] - init_value) / init_value

    # 获取基准指数(benchmark)在回测期间的历史数据
    bm_df = attribute_daterange_history(context.benchmark, context.start_date, context.end_date)
    # 获取基准指数在回测开始时的开盘价，并将其存储在bm_init变量中
    bm_init = bm_df['open'][0]
    # 计算基准(Benchmark)收益率
    plt_df['benchmark_ratio'] = (bm_df['open'] - bm_init) / bm_init

    # 可视化
    #设置字体 显示汉字
    plt.rcParams["font.sans-serif"]="SimHei"
    #用来正常显示负号
    plt.rcParams['axes.unicode_minus']=False
    #设置画布的尺寸为18*10
    plt.figure(figsize = (18,10))
    plt.title("python SMA量化框架")
    #绘制收益率曲线
    plt.plot(plt_df['ratio'], label = "ratio")
    # 绘制基准收益率曲线
    plt.plot(plt_df['benchmark_ratio'], label = "benchmark_ratio")
    #设置x轴标签
    plt.xlabel("日期")
    #设置y轴标签
    plt.ylabel("收益率")
    #x坐标斜率
    plt.xticks(rotation=46)
    # 添加图注
    plt.legend()
    # 显示
    plt.show()

    #初始化函数，设定基准等等
    #initialize(context)函数接受一个context参数，这个参数就是一个Context类的实例。
    #在函数内部，可以使用这个context对象来访问其属性和方法，以实现特定的功能。
    def initialize(context):
        # 设定002624作为基准
        set_bench_mark('000001.SZ')
        # 设定10日均线全局参数
        g.ma1 = 10
        # 设定60日均线全局参数
        g.ma2 = 60
        # 设定要操作的股票002572
        g.security = '002572.SZ'

    #该函数每个bar(单位时间)会调用一次
    def handle_data(context):
        # 获取历史数据
        hist = attribute_history(g.security, g.ma2)
        # 10日均线
        ma10 = hist['close'][-g.ma1:].mean()
        # 60日均线
        ma60 = hist['close'].mean()
        # 如果10日均线大于60日均线，并且没有持仓，则全仓买入
        if ma10 > ma60 and g.security not in context.positions:
            order_value(g.security, context.cash)
        # 如果10日均线小于60日均线，并且持仓，则清仓
        elif ma10 < ma60 and g.security in context.positions:
            order_target(g.security, 0)


run()